In [1]:
import numpy as np
import scipy.sparse
from scipy.sparse.linalg import lobpcg, eigsh, minres, LinearOperator
from scipy.sparse import csr_matrix
import time
from utils.tools import build_weighted_bethe_hessian
from scipy.sparse.csgraph import minimum_spanning_tree

In [2]:
adj_path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.adj"
adjacency_matrix = scipy.sparse.load_npz(adj_path + ".npz")
adjacency_matrix.data = adjacency_matrix.data ** 0.3
degrees = np.asarray(adjacency_matrix.sum(axis=1)).flatten()

In [3]:
r = np.sqrt(np.mean(degrees**2) / np.mean(degrees) - 1)
#r = np.mean(adjacency_matrix.data**2)
D, A = build_weighted_bethe_hessian(adjacency_matrix, r)

n = adjacency_matrix.shape[0]
I = scipy.sparse.eye(n, format='csr')
Hr = D - A #+ I * np.mean(degrees)

In [5]:
class MatrixOperator(object):

    def __init__(self, A):
        self.A = A.astype(PETSc.ScalarTye)
        self.n_calls = 0

    def mult(self, A, x, y):
        xx = x.getArray(readonly=1)
        yy = y.getArray(readonly=0)
        yy[:] = self.A.dot(xx)
        self.n_calls += 1
        
    def getDiagonal(self, A, y):
        yy = y.getArray(readonly=0)
        yy[:] = self.A.diagonal()

In [20]:
from petsc4py import PETSc
from slepc4py import SLEPc

A = Hr
k = 100
tol = 1e-2
max_iter = 100

### Setup matrix operator
n = A.shape[0]
mat = MatrixOperator(A)
A_operator = PETSc.Mat().createPython([n, n], mat)
A_operator.setUp()

### Solve eigenproblem
E = SLEPc.EPS()
E.create()
E.setOperators(A_operator)
E.setProblemType(SLEPc.EPS.ProblemType.HEP)
#E.setType(SLEPc.EPS.Type.JD)
E.setDimensions(k)
E.setTolerances(tol, max_iter)
E.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL)

def monitor(eps, its, nconv, eig, err):
    print(its, nconv)
    print(err)

E.setMonitor(monitor)

start = time.time()
E.solve()
print("Time_elapsed: %d" % (time.time() - start))
print("Number of calls to Ax: %d" % mat.n_calls)

1 32
[7.31836443756012e-19, 2.565367152620394e-17, 4.644127367257285e-17, 4.2435131282434645e-19, 5.671615117577661e-17, 4.9874238700791083e-17, 2.264431195411471e-16, 3.0952614426837197e-16, 7.825913569279251e-18, 1.8403941923724853e-16, 1.8999946617735898e-16, 4.694405036957613e-16, 1.5619374250250367e-16, 9.778759310358618e-17, 4.4734210794824625e-16, 3.31307096186566e-16, 1.6043394020799484e-15, 1.2211843934731613e-15, 5.718787315729648e-16, 4.489062245865606e-16, 4.823610112949529e-14, 5.889189549677915e-12, 7.500257588947473e-10, 6.795573291788536e-10, 1.3755147351439517e-08, 3.060298244880906e-07, 3.72732042909934e-05, 0.00044527956158122714, 0.00013189893207911333, 0.0007112920068847421, 0.002108565409276747, 0.0030276061841620363, 0.018853783228598143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

6 66
[7.31836443756012e-19, 2.565367152620394e-17, 4.644127367257285e-17, 4.2435131282434645e-19, 5.671615117577661e-17, 4.9874238700791083e-17, 2.264431195411471e-16, 3.0952614426837197e-16, 7.825913569279251e-18, 1.8403941923724853e-16, 1.8999946617735898e-16, 4.694405036957613e-16, 1.5619374250250367e-16, 9.778759310358618e-17, 4.4734210794824625e-16, 3.31307096186566e-16, 1.6043394020799484e-15, 1.2211843934731613e-15, 5.718787315729648e-16, 4.489062245865606e-16, 4.823610112949529e-14, 5.889189549677915e-12, 7.500257588947473e-10, 6.795573291788536e-10, 1.3755147351439517e-08, 3.060298244880906e-07, 3.72732042909934e-05, 0.00044527956158122714, 0.00013189893207911333, 0.0007112920068847421, 0.002108565409276747, 0.0030276061841620363, 2.811237670310726e-06, 1.9606569962057206e-06, 1.5744474172291996e-06, 4.480397682143197e-05, 0.00018307045272932733, 0.00017606782257454412, 0.006136641906613136, 3.789344418303768e-07, 1.2794973768700917e-06, 8.319899624546852e-07, 1.66268566771845

10 83
[7.31836443756012e-19, 2.565367152620394e-17, 4.644127367257285e-17, 4.2435131282434645e-19, 5.671615117577661e-17, 4.9874238700791083e-17, 2.264431195411471e-16, 3.0952614426837197e-16, 7.825913569279251e-18, 1.8403941923724853e-16, 1.8999946617735898e-16, 4.694405036957613e-16, 1.5619374250250367e-16, 9.778759310358618e-17, 4.4734210794824625e-16, 3.31307096186566e-16, 1.6043394020799484e-15, 1.2211843934731613e-15, 5.718787315729648e-16, 4.489062245865606e-16, 4.823610112949529e-14, 5.889189549677915e-12, 7.500257588947473e-10, 6.795573291788536e-10, 1.3755147351439517e-08, 3.060298244880906e-07, 3.72732042909934e-05, 0.00044527956158122714, 0.00013189893207911333, 0.0007112920068847421, 0.002108565409276747, 0.0030276061841620363, 2.811237670310726e-06, 1.9606569962057206e-06, 1.5744474172291996e-06, 4.480397682143197e-05, 0.00018307045272932733, 0.00017606782257454412, 0.006136641906613136, 3.789344418303768e-07, 1.2794973768700917e-06, 8.319899624546852e-07, 1.6626856677184

14 92
[7.31836443756012e-19, 2.565367152620394e-17, 4.644127367257285e-17, 4.2435131282434645e-19, 5.671615117577661e-17, 4.9874238700791083e-17, 2.264431195411471e-16, 3.0952614426837197e-16, 7.825913569279251e-18, 1.8403941923724853e-16, 1.8999946617735898e-16, 4.694405036957613e-16, 1.5619374250250367e-16, 9.778759310358618e-17, 4.4734210794824625e-16, 3.31307096186566e-16, 1.6043394020799484e-15, 1.2211843934731613e-15, 5.718787315729648e-16, 4.489062245865606e-16, 4.823610112949529e-14, 5.889189549677915e-12, 7.500257588947473e-10, 6.795573291788536e-10, 1.3755147351439517e-08, 3.060298244880906e-07, 3.72732042909934e-05, 0.00044527956158122714, 0.00013189893207911333, 0.0007112920068847421, 0.002108565409276747, 0.0030276061841620363, 2.811237670310726e-06, 1.9606569962057206e-06, 1.5744474172291996e-06, 4.480397682143197e-05, 0.00018307045272932733, 0.00017606782257454412, 0.006136641906613136, 3.789344418303768e-07, 1.2794973768700917e-06, 8.319899624546852e-07, 1.6626856677184

In [ ]:
### Collect results
print("")
its = E.getIterationNumber()
print("Number of iterations of the method: %i" % its)
sol_type = E.getType()
print("Solution method: %s" % sol_type)
nev, ncv, mpd = E.getDimensions()
print("Number of requested eigenvalues: %i" % nev)
tol, maxit = E.getTolerances()
print("Stopping condition: tol=%.4g, maxit=%d" % (tol, maxit))
nconv = E.getConverged()
print("Number of converged eigenpairs: %d" % nconv)
nconv = min(nconv, k)

if nconv < k:
    raise ZeroDivisionError("Failed to converge for requested number of k with maxiter=%d" % max_iter)

vecs = np.zeros([n, nconv])
vals = np.zeros(nconv)

xr, tmp = A_operator.getVecs()
xi, tmp = A_operator.getVecs()

if nconv > 0:
    for i in range(nconv):
        val = E.getEigenpair(i, xr, xi)
        vals[i] = val.real
        vecs[:, i] = xr

In [9]:
vals

array([-52.81019524, -11.47311405,  -7.32714123,  -5.68404989,
        -5.01421744,  -4.6225821 ,  -4.37988141,  -3.95041617,
        -3.50948522,  -3.31424227])

In [12]:
vals

array([-5.28101952e+01, -1.14731145e+01, -7.32714200e+00, -5.68407192e+00,
       -5.01466348e+00, -4.62261124e+00, -4.37962547e+00, -3.95044248e+00,
       -3.50956361e+00, -3.31421900e+00, -3.17637015e+00, -2.69915527e+00,
       -2.59356620e+00, -2.35605036e+00, -2.32639601e+00, -2.00888795e+00,
       -1.85249372e+00, -1.80079322e+00, -1.71902310e+00, -1.66647307e+00,
       -1.53121772e+00, -1.45979640e+00, -1.39324024e+00, -1.35781663e+00,
       -1.29056868e+00, -1.22502130e+00, -1.17121063e+00, -1.12776150e+00,
       -1.09565394e+00, -1.06910070e+00, -1.01113585e+00, -9.83283988e-01,
       -9.20765552e-01, -9.13510630e-01, -8.87122342e-01, -8.51867882e-01,
       -8.18154868e-01, -7.82247782e-01, -7.46196034e-01, -7.25473643e-01,
       -7.20940108e-01, -7.04505434e-01, -6.68241236e-01, -6.29097033e-01,
       -6.22349114e-01, -5.86517676e-01, -5.64382177e-01, -5.62295884e-01,
       -5.41607765e-01, -5.17848393e-01, -4.97719222e-01, -4.79210838e-01,
       -4.72909313e-01, -

In [18]:
output_path = "../data/wiki/win=1_correctly_weighted_bethe_hessian_fast_slepc_pow=0.3_dim=100"
np.save(output_path + ".vecs", vecs[:,:100])
np.save(output_path + ".vals", vals[:100])
np.save(output_path + ".degrees", degrees)

In [19]:
import shutil
base_path = '../data/wiki/'
shutil.copyfile(base_path + 'win=1_weighted_bethe_hessian_slepc_scaled_abs_tol=1e-3_pow=0.3_dim=500.words.vocab', 
                base_path + 'win=1_correctly_weighted_bethe_hessian_fast_slepc_pow=0.3_dim=100' + ".words.vocab")

'../data/wiki/win=1_correctly_weighted_bethe_hessian_fast_slepc_pow=0.3_dim=100.words.vocab'